In [1]:
import numpy as np

In [2]:
s = 64
f = 128

data = np.random.rand(s, s, f, f).astype(np.float32)

mask_1 = np.zeros(shape=(f, f), dtype=np.float32)
mask_1[(f * 5) // 10:(f * 6) // 10, (f * 2) // 10:(f * 3)// 10] = 1
boolean_mask_1 = mask_1 == 1

mask_2 = np.zeros(shape=(f, f), dtype=np.float32)
mask_2[(f * 2) // 10:(f * 7) // 10, (f * 1) // 10:(f * 8)// 10] = 1
boolean_mask_2 = mask_2 == 1

In [3]:
result_1 = np.zeros(shape=(s*s), dtype=np.float32)
result_2 = np.zeros(shape=(s*s), dtype=np.float32)

data_list = data.reshape((-1, f, f))

In [4]:
%timeit result_1[:] = data_list[:, boolean_mask_1].std(axis=1)

%timeit result_2[:] = data_list[:, boolean_mask_2].std(axis=1)

true_result_1 = result_1.copy()
true_result_2 = result_2.copy()

29.2 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.07 s ± 42.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
def single(result, data_list, boolean_mask):
    for i in range(len(result)):
        view = data_list[i, boolean_mask]
        result[i] = view.std()

result_1[:] = np.zeros(shape=(s*s), dtype=np.float32)
result_2[:] = np.zeros(shape=(s*s), dtype=np.float32)
        
%timeit single(result_1, data_list, boolean_mask_1)
%timeit single(result_2, data_list, boolean_mask_2)

assert np.allclose(result_1, true_result_1)
assert np.allclose(result_2, true_result_2)

503 ms ± 32.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
814 ms ± 33.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
def tiled(result, data_list, boolean_mask, tilesize):
    repeats = len(result) // tilesize
    tail = len(result) % tilesize
    for i in range(repeats):
        result[i*tilesize:(i+1)*tilesize] = data_list[i*tilesize:(i+1)*tilesize, boolean_mask].std(axis=1)
    if tail != 0:
        result[-tail:] = data_list[-tail:, boolean_mask].std(axis=1)

result_1[:] = np.zeros(shape=(s*s), dtype=np.float32)
result_2[:] = np.zeros(shape=(s*s), dtype=np.float32)
        
%timeit tiled(result_1, data_list, boolean_mask_1, 32)
%timeit tiled(result_2, data_list, boolean_mask_2, 32)

assert np.allclose(result_1, true_result_1)
assert np.allclose(result_2, true_result_2)

28.6 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
519 ms ± 52.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
